In [1]:
# Example of the autograd engine at work:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x0000022C563F4850>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x0000022C563F4940>


In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.3305, 0.2463, 0.0022],
        [0.3305, 0.2463, 0.0022],
        [0.3305, 0.2463, 0.0022],
        [0.3305, 0.2463, 0.0022],
        [0.3305, 0.2463, 0.0022]])
tensor([0.3305, 0.2463, 0.0022])


In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [7]:
# Indentically
z = torch.matmul(x, w)+b
print(z.requires_grad)
z_det = z.detach()
print(z_det.requires_grad)

True
False


In [8]:
# We do this mainly to speed up computations if we are just using the neural net (not training and doing back-prop)
# We also do this when we want to FREEZE parameters and finetune and pretrained network

In [9]:
# If we have a multidimensional loss function, a vector valued one to be more specific, then we will need the Jacobian/Jacobian product instead
# As an example:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print('First call\n', inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print('Second call\n', inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


In [10]:
# As given in the tutorial, a very useful explanation of the function syntax: """Previously we were calling backward() function without parameters. This is essentially equivalent to calling backward(torch.tensor(1.0)), which is a useful way to compute the gradients in case of a scalar-valued function, such as loss during neural network training."""